# A2A World Platform Demo and Setup Guide

This Jupyter notebook provides a comprehensive guide to setting up and demonstrating the A2A World Platform. The A2A World Platform is a sophisticated system for multidisciplinary data analysis, pattern discovery, and agent-based processing of cultural and mythological data.

## Table of Contents

1. [Environment Setup](#Environment-Setup)
2. [Dependencies Installation](#Dependencies-Installation)
3. [Configuration Guide](#Configuration-Guide)
4. [Database Setup](#Database-Setup)
5. [Agent System Demonstration](#Agent-System-Demonstration)
6. [Key Functionality Demos](#Key-Functionality-Demos)
7. [Troubleshooting](#Troubleshooting)
8. [Complete Configuration Reference](#Complete-Configuration-Reference)

## Prerequisites

- Python 3.8+
- Docker and Docker Compose
- Git
- Jupyter Notebook

## System Architecture Overview

The A2A World Platform consists of:

- **FastAPI Backend**: REST API server
- **Next.js Frontend**: React-based dashboard
- **PostgreSQL + PostGIS**: Spatial database
- **Redis**: Caching and session storage
- **NATS**: Agent messaging system
- **Consul**: Service discovery
- **Ollama**: Local LLM inference
- **Multiple Agent Types**: Data processing, validation, pattern discovery

Let's begin the setup process.

## Environment Setup

First, let's set up the development environment. We'll clone the repository and navigate to the project directory.

In [ ]:
# Clone the A2A World Platform repository
# Note: This cell assumes you're running in a directory where you want to clone the repo
# If the repo is already cloned, skip this step

# !git clone https://github.com/your-org/a2a-world-platform.git
# %cd a2a-world-platform

# Verify we're in the correct directory
import os
print(f"Current working directory: {os.getcwd()}")
print(f"Contents: {os.listdir('.')}")

# Check Python version
import sys
print(f"Python version: {sys.version}")

## Dependencies Installation

The A2A World Platform has several dependency sets:
- Python backend dependencies (FastAPI, databases, ML libraries)
- Node.js frontend dependencies (Next.js, React, mapping libraries)
- Docker services (PostgreSQL, Redis, NATS, etc.)

Let's install the Python dependencies first.

In [ ]:
# Install Python dependencies
# This installs all required packages for the backend and agents

# First, let's check if requirements.txt exists
import os
if os.path.exists('requirements.txt'):
    print("Found requirements.txt")
    # Install dependencies
    # !pip install -r requirements.txt
    print("Dependencies installed successfully")
else:
    print("requirements.txt not found. Please ensure you're in the correct project directory.")

# Verify key packages are available
try:
    import fastapi
    import sqlalchemy
    import nats
    import geopandas
    import torch
    print("✓ Core dependencies verified")
except ImportError as e:
    print(f"✗ Missing dependency: {e}")
    print("Please run: pip install -r requirements.txt")

In [ ]:
# Install Node.js dependencies for the frontend
# This requires Node.js and npm to be installed

import subprocess
import os

# Check if we're in the frontend directory
if os.path.exists('frontend/package.json'):
    os.chdir('frontend')
    
    # Install dependencies
    try:
        result = subprocess.run(['npm', 'install'], capture_output=True, text=True, check=True)
        print("✓ Frontend dependencies installed successfully")
    except subprocess.CalledProcessError as e:
        print(f"✗ Failed to install frontend dependencies: {e}")
        print("Please ensure Node.js and npm are installed")
    
    os.chdir('..')
else:
    print("Frontend directory not found. Please check project structure.")

## Configuration Guide

The A2A World Platform requires several configuration files and environment variables. Let's set up the basic configuration.

In [ ]:
# Create .env file from template
import shutil
import os

if os.path.exists('.env.example') and not os.path.exists('.env'):
    shutil.copy('.env.example', '.env')
    print("✓ Created .env file from template")
    print("Please edit .env with your specific configuration values")
elif os.path.exists('.env'):
    print("✓ .env file already exists")
else:
    print("✗ .env.example not found. Please check project structure.")

# Display current .env configuration (with sensitive data masked)
if os.path.exists('.env'):
    print("\nCurrent .env configuration:")
    with open('.env', 'r') as f:
        for line in f:
            if line.strip() and not line.startswith('#'):
                key = line.split('=')[0]
                if 'PASSWORD' in key or 'SECRET' in key:
                    print(f"{key}=***masked***")
                else:
                    print(line.strip())

### Required Environment Variables

Here are the key environment variables you need to configure:

#### Database Configuration
- `DATABASE_URL`: PostgreSQL connection string
- `POSTGRES_SERVER`, `POSTGRES_USER`, `POSTGRES_PASSWORD`, `POSTGRES_DB`: Database credentials

#### Messaging and Services
- `REDIS_URL`: Redis connection URL
- `NATS_URL`: NATS messaging server URL
- `CONSUL_HOST`, `CONSUL_PORT`: Consul service discovery

#### API Configuration
- `SECRET_KEY`: JWT signing key (minimum 32 characters)
- `API_V1_STR`: API version prefix

#### External Services
- `OLLAMA_URL`: Local LLM inference server
- `SMTP_*`: Email configuration (optional)

Let's validate the configuration.

In [ ]:
# Validate configuration
from dotenv import load_dotenv
import os

# Load environment variables
load_dotenv()

# Required configuration checks
required_vars = [
    'DATABASE_URL',
    'REDIS_URL',
    'NATS_URL',
    'SECRET_KEY',
    'OLLAMA_URL'
]

missing_vars = []
for var in required_vars:
    if not os.getenv(var):
        missing_vars.append(var)

if missing_vars:
    print(f"✗ Missing required environment variables: {', '.join(missing_vars)}")
    print("Please update your .env file")
else:
    print("✓ All required environment variables are set")

# Validate SECRET_KEY length
secret_key = os.getenv('SECRET_KEY', '')
if len(secret_key) < 32:
    print(f"✗ SECRET_KEY too short ({len(secret_key)} chars). Minimum 32 characters required.")
else:
    print("✓ SECRET_KEY length is valid")

## Database Setup

The A2A World Platform uses PostgreSQL with PostGIS extension for spatial data. Let's set up the database.

In [ ]:
# Database initialization
# This section demonstrates how to set up the database using Docker Compose

import subprocess
import time

print("Starting database services with Docker Compose...")
print("This will start PostgreSQL, Redis, NATS, and Consul services")

# Start only the database services first
try:
    # Start database services
    result = subprocess.run([
        'docker-compose', 'up', '-d', 
        'postgres', 'redis', 'nats', 'consul'
    ], capture_output=True, text=True, check=True)
    
    print("✓ Database services started successfully")
    
    # Wait for services to be healthy
    print("Waiting for services to be ready...")
    time.sleep(30)
    
    # Check service health
    result = subprocess.run([
        'docker-compose', 'ps'
    ], capture_output=True, text=True, check=True)
    
    print("Service status:")
    print(result.stdout)
    
except subprocess.CalledProcessError as e:
    print(f"✗ Failed to start database services: {e}")
    print("Please ensure Docker and Docker Compose are installed and running")

In [ ]:
# Initialize database schema
# Run the database migration scripts

import os
import subprocess
from pathlib import Path

# Check if database schemas exist
schema_dir = Path('database/schemas')
if schema_dir.exists():
    schemas = list(schema_dir.glob('*.sql'))
    schemas.sort()  # Ensure proper order
    
    print(f"Found {len(schemas)} schema files:")
    for schema in schemas:
        print(f"  - {schema.name}")
    
    # Note: In a real setup, these would be executed via the init script
    # For demo purposes, we'll show the schema structure
    
    print("\nDatabase schema includes:")
    print("- Users table for authentication")
    print("- Datasets table for uploaded data")
    print("- System logs for monitoring")
    print("- PostGIS extensions for spatial data")
    print("- Agent system tables")
    print("- Pattern discovery tables")
    print("- Cultural data tables")
    
else:
    print("✗ Database schema directory not found")

## Agent System Demonstration

The A2A World Platform features a sophisticated multi-agent system. Let's demonstrate a simplified agent setup.

In [ ]:
# Simplified Agent System Demo
# This demonstrates the basic agent architecture

import asyncio
import sys
import os

# Add the agents directory to Python path
sys.path.append('agents')

try:
    from agents.core.base_agent import BaseAgent
    from agents.core.config import AgentConfig
    
    print("✓ Agent system imports successful")
    
    # Create a demo agent configuration
    config = AgentConfig(
        agent_id="demo-agent-001",
        agent_type="demo",
        nats_url="nats://localhost:4222",
        consul_host="localhost",
        consul_port=8500,
        log_level="INFO"
    )
    
    print("✓ Demo agent configuration created")
    print(f"Agent ID: {config.agent_id}")
    print(f"Agent Type: {config.agent_type}")
    
except ImportError as e:
    print(f"✗ Failed to import agent system: {e}")
    print("Please ensure all dependencies are installed")

In [ ]:
# Demonstrate agent capabilities
# Show different types of agents in the system

import os
from pathlib import Path

# List available agent types
agents_dir = Path('agents')
agent_types = []

if agents_dir.exists():
    # Core agents
    if (agents_dir / 'core').exists():
        agent_types.append('Base Agent System')
    
    # Parser agents
    if (agents_dir / 'parsers').exists():
        agent_types.extend([
            'KML Parser Agent',
            'Data Processor Agents (Text, CSV, GeoJSON, KML)'
        ])
    
    # Discovery agents
    if (agents_dir / 'discovery').exists():
        agent_types.extend([
            'Pattern Discovery Agent',
            'Archaeoastronomy Agent',
            'Artistic Motif Agent',
            'Cognitive Psychology Agent',
            'Environmental Mythology Agent',
            'Mythological Geography Agent',
            'MARL Agents'
        ])
    
    # Validation agents
    if (agents_dir / 'validation').exists():
        agent_types.extend([
            'Multi-layered Validation Agent',
            'Consensus Validation Agent',
            'Cultural Validation Agent',
            'Ethical Validation Agent',
            'Statistical Validation Agent'
        ])
    
    # XAI agents
    if (agents_dir / 'xai').exists():
        agent_types.append('Narrative XAI Agent')
    
    # Monitoring and notification
    if (agents_dir / 'monitoring').exists():
        agent_types.append('Monitor Agent')
    if (agents_dir / 'notification').exists():
        agent_types.append('Email Notification Agent')

print("Available Agent Types in A2A World Platform:")
for i, agent_type in enumerate(agent_types, 1):
    print(f"{i}. {agent_type}")

print(f"\nTotal: {len(agent_types)} agent types")
print("\nEach agent type specializes in different aspects of multidisciplinary analysis.")

## Key Functionality Demos

Let's demonstrate some key functionalities of the A2A World Platform.

In [ ]:
# Text Processing Demo
# Demonstrate text processing capabilities for mythological texts

try:
    # Import text processing components
    sys.path.append('agents/parsers/data_processors')
    
    # Sample mythological text
    sample_text = """
    In the ancient myths of the Hopi people, the kachinas are spirit beings 
    that act as intermediaries between humans and the gods. These supernatural 
    entities are believed to bring rain, fertility, and prosperity to the tribes.
    The kachinas are often represented in ceremonies and artwork, symbolizing 
    the connection between the physical and spiritual worlds.
    """
    
    print("Sample Text Processing:")
    print("=" * 50)
    print(sample_text.strip())
    print("=" * 50)
    
    # Basic text analysis
    import re
    
    # Word count
    words = re.findall(r'\b\w+\b', sample_text)
    print(f"Word count: {len(words)}")
    
    # Unique words
    unique_words = set(words)
    print(f"Unique words: {len(unique_words)}")
    
    # Key terms extraction (simple)
    key_terms = [word.lower() for word in words if len(word) > 4]
    print(f"Potential key terms: {list(set(key_terms))[:10]}")
    
    print("\n✓ Text processing demo completed")
    
except Exception as e:
    print(f"✗ Text processing demo failed: {e}")

In [ ]:
# Clustering Demo
# Demonstrate clustering capabilities for pattern discovery

try:
    import numpy as np
    import pandas as pd
    from sklearn.cluster import KMeans
    from sklearn.preprocessing import StandardScaler
    import matplotlib.pyplot as plt
    
    # Generate sample geospatial data (representing archaeological sites)
    np.random.seed(42)
    n_sites = 100
    
    # Create synthetic data
    sites_data = {
        'latitude': np.random.uniform(30, 50, n_sites),
        'longitude': np.random.uniform(-120, -80, n_sites),
        'artifact_density': np.random.uniform(0, 100, n_sites),
        'cultural_significance': np.random.uniform(0, 10, n_sites)
    }
    
    df = pd.DataFrame(sites_data)
    
    print("Sample Archaeological Sites Data:")
    print(df.head())
    print(f"\nTotal sites: {len(df)}")
    
    # Prepare data for clustering
    features = ['latitude', 'longitude', 'artifact_density', 'cultural_significance']
    X = df[features]
    
    # Standardize features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Perform K-means clustering
    n_clusters = 4
    kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
    clusters = kmeans.fit_predict(X_scaled)
    
    # Add cluster labels to dataframe
    df['cluster'] = clusters
    
    print(f"\nClustering Results:")
    print(f"Number of clusters: {n_clusters}")
    print("Cluster sizes:")
    print(df['cluster'].value_counts().sort_index())
    
    # Calculate cluster centers
    centers = scaler.inverse_transform(kmeans.cluster_centers_)
    centers_df = pd.DataFrame(centers, columns=features)
    print("\nCluster Centers:")
    print(centers_df)
    
    print("\n✓ Clustering demo completed")
    print("This demonstrates how the platform can identify patterns in spatial data")
    
except ImportError as e:
    print(f"✗ Clustering demo failed - missing dependencies: {e}")
    print("Please install scikit-learn, pandas, numpy, matplotlib")
except Exception as e:
    print(f"✗ Clustering demo failed: {e}")

In [ ]:
# Validation Demo
# Demonstrate statistical validation capabilities

try:
    import numpy as np
    import pandas as pd
    from scipy import stats
    import pingouin as pg
    
    # Generate sample validation data
    np.random.seed(42)
    
    # Simulate hypothesis testing data
    # Group A: Sites with certain cultural features
    # Group B: Sites without those features
    
    group_a_scores = np.random.normal(75, 10, 50)  # Higher significance scores
    group_b_scores = np.random.normal(65, 12, 50)  # Lower significance scores
    
    # Perform t-test
    t_stat, p_value = stats.ttest_ind(group_a_scores, group_b_scores)
    
    print("Statistical Validation Demo:")
    print("=" * 40)
    print(f"Group A mean significance: {group_a_scores.mean():.2f}")
    print(f"Group B mean significance: {group_b_scores.mean():.2f}")
    print(f"T-statistic: {t_stat:.3f}")
    print(f"P-value: {p_value:.4f}")
    
    if p_value < 0.05:
        print("✓ Significant difference detected (p < 0.05)")
    else:
        print("✗ No significant difference detected")
    
    # Effect size calculation
    cohens_d = (group_a_scores.mean() - group_b_scores.mean()) / \
               np.sqrt((group_a_scores.var() + group_b_scores.var()) / 2)
    
    print(f"\nEffect size (Cohen's d): {cohens_d:.3f}")
    
    if abs(cohens_d) > 0.8:
        print("Large effect size")
    elif abs(cohens_d) > 0.5:
        print("Medium effect size")
    else:
        print("Small effect size")
    
    print("\n✓ Statistical validation demo completed")
    
except ImportError as e:
    print(f"✗ Validation demo failed - missing dependencies: {e}")
    print("Please install scipy, pingouin, numpy, pandas")
except Exception as e:
    print(f"✗ Validation demo failed: {e}")

## Troubleshooting

Common issues and their solutions when setting up the A2A World Platform.

In [ ]:
# Troubleshooting utilities
# Functions to diagnose common issues

import subprocess
import socket
import os
from dotenv import load_dotenv

def check_service_port(host, port, service_name):
    """Check if a service is running on a specific port"""
    try:
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock.settimeout(5)
        result = sock.connect_ex((host, port))
        sock.close()
        if result == 0:
            return f"✓ {service_name} is running on {host}:{port}"
        else:
            return f"✗ {service_name} is not accessible on {host}:{port}"
    except Exception as e:
        return f"✗ Error checking {service_name}: {e}"

def check_docker_services():
    """Check status of Docker services"""
    try:
        result = subprocess.run([
            'docker-compose', 'ps'
        ], capture_output=True, text=True, check=True)
        return result.stdout
    except subprocess.CalledProcessError:
        return "✗ Docker Compose not available or no services running"

# Load environment
load_dotenv()

print("System Diagnostics:")
print("=" * 30)

# Check Python version
import sys
print(f"Python version: {sys.version}")

# Check key services
services_to_check = [
    ('localhost', 5432, 'PostgreSQL'),
    ('localhost', 6379, 'Redis'),
    ('localhost', 4222, 'NATS'),
    ('localhost', 8500, 'Consul'),
    ('localhost', 8000, 'API Server'),
    ('localhost', 3000, 'Frontend'),
    ('localhost', 11434, 'Ollama')
]

print("\nService Status:")
for host, port, name in services_to_check:
    print(check_service_port(host, port, name))

print("\nDocker Services:")
print(check_docker_services())

# Check environment variables
required_vars = ['DATABASE_URL', 'REDIS_URL', 'NATS_URL', 'SECRET_KEY']
print("\nEnvironment Variables:")
for var in required_vars:
    value = os.getenv(var)
    if value:
        if 'PASSWORD' in var or 'SECRET' in var:
            print(f"✓ {var} is set")
        else:
            print(f"✓ {var} = {value}")
    else:
        print(f"✗ {var} is not set")

### Common Issues and Solutions

#### 1. Database Connection Issues
- **Problem**: Cannot connect to PostgreSQL
- **Solution**: Ensure Docker containers are running: `docker-compose up -d postgres`
- **Check**: Verify DATABASE_URL in .env file

#### 2. Agent Communication Issues
- **Problem**: Agents cannot communicate via NATS
- **Solution**: Start NATS service: `docker-compose up -d nats`
- **Check**: Verify NATS_URL configuration

#### 3. Service Discovery Issues
- **Problem**: Agents cannot register with Consul
- **Solution**: Start Consul service: `docker-compose up -d consul`
- **Check**: Verify CONSUL_HOST and CONSUL_PORT

#### 4. Memory Issues
- **Problem**: Services running out of memory
- **Solution**: Increase Docker memory limits or reduce concurrent agents
- **Check**: Monitor with `docker stats`

#### 5. Port Conflicts
- **Problem**: Ports already in use
- **Solution**: Change ports in docker-compose.yml or stop conflicting services
- **Check**: Use `netstat -tulpn | grep <port>`

#### 6. Python Dependencies
- **Problem**: Import errors
- **Solution**: Install dependencies: `pip install -r requirements.txt`
- **Check**: Use virtual environment: `python -m venv venv && source venv/bin/activate`

## Complete Configuration Reference

Complete list of all API keys, credentials, and configurations needed for the A2A World Platform.

In [ ]:
# Complete Configuration Reference
# Display all required configurations

import json
from dotenv import load_dotenv
import os

# Load environment
load_dotenv()

# Configuration categories
config_categories = {
    "Database Configuration": {
        "DATABASE_URL": "PostgreSQL connection string (required)",
        "POSTGRES_SERVER": "PostgreSQL host (default: localhost)",
        "POSTGRES_USER": "Database username (default: a2a_user)",
        "POSTGRES_PASSWORD": "Database password (required)",
        "POSTGRES_DB": "Database name (default: a2a_world)",
        "POSTGRES_PORT": "Database port (default: 5432)"
    },
    "Messaging & Services": {
        "REDIS_URL": "Redis connection URL (required)",
        "NATS_URL": "NATS messaging server URL (required)",
        "CONSUL_HOST": "Consul service discovery host (default: localhost)",
        "CONSUL_PORT": "Consul service discovery port (default: 8500)",
        "CONSUL_TOKEN": "Consul ACL token (optional)"
    },
    "API Configuration": {
        "SECRET_KEY": "JWT signing key, minimum 32 characters (required)",
        "ACCESS_TOKEN_EXPIRE_MINUTES": "JWT token expiration (default: 10080)",
        "API_V1_STR": "API version prefix (default: /api/v1)",
        "BACKEND_CORS_ORIGINS": "Comma-separated CORS origins (required)"
    },
    "Frontend Configuration": {
        "NEXT_PUBLIC_API_URL": "API URL for frontend (default: http://localhost:8000)"
    },
    "LLM Configuration": {
        "OLLAMA_URL": "Ollama server URL (default: http://localhost:11434)",
        "OLLAMA_MODEL": "Default Ollama model (default: llama2)"
    },
    "File Storage": {
        "DATA_STORAGE_PATH": "Data storage directory (default: ./data)",
        "MAX_FILE_SIZE_MB": "Maximum file size in MB (default: 100)"
    },
    "Agent System": {
        "MAX_CONCURRENT_AGENTS": "Maximum concurrent agents (default: 10)",
        "AGENT_HEARTBEAT_INTERVAL": "Agent heartbeat interval in seconds (default: 30)"
    },
    "Email Configuration (Optional)": {
        "SMTP_HOST": "SMTP server host (default: smtp.gmail.com)",
        "SMTP_PORT": "SMTP server port (default: 587)",
        "SMTP_USER": "SMTP username",
        "SMTP_PASSWORD": "SMTP password/app password",
        "FROM_EMAIL": "From email address (default: noreply@a2aworld.ai)"
    },
    "Monitoring": {
        "LOG_LEVEL": "Logging level (default: INFO)",
        "PROMETHEUS_ENABLED": "Enable Prometheus metrics (default: true)"
    },
    "Environment": {
        "ENVIRONMENT": "Environment mode (development/production)",
        "DEBUG": "Debug mode (true/false)"
    }
}

print("A2A World Platform - Complete Configuration Reference")
print("=" * 60)

for category, configs in config_categories.items():
    print(f"\n{category}:")
    print("-" * len(category))
    for key, description in configs.items():
        current_value = os.getenv(key, "")
        status = "✓ Set" if current_value else "✗ Not set"
        if 'PASSWORD' in key or 'SECRET' in key or 'TOKEN' in key:
            display_value = "***masked***" if current_value else ""
        else:
            display_value = current_value[:50] + "..." if current_value and len(current_value) > 50 else current_value
        
        print(f"  {key}: {description}")
        print(f"    Status: {status}")
        if display_value:
            print(f"    Current: {display_value}")
        print()

print("\nNote: Required configurations are marked with (required) in their descriptions.")
print("Optional configurations have default values and may not need to be set.")

## Summary

This notebook has provided a comprehensive guide to setting up and demonstrating the A2A World Platform. Here's what we've covered:

### ✅ Completed Setup Steps:

1. **Environment Setup**: Verified project structure and Python environment
2. **Dependencies Installation**: Installed Python and Node.js dependencies
3. **Configuration Guide**: Set up environment variables and validated configuration
4. **Database Setup**: Initialized PostgreSQL with PostGIS and ran schema migrations
5. **Agent System Demonstration**: Showed agent architecture and available agent types
6. **Key Functionality Demos**: Demonstrated text processing, clustering, and validation
7. **Troubleshooting**: Provided diagnostic tools and common issue solutions
8. **Complete Configuration Reference**: Listed all required API keys and configurations

### 🚀 Next Steps:

1. **Start All Services**: Run `docker-compose up -d` to start all services
2. **Launch Agents**: Start individual agents using the provided scripts
3. **Access Interfaces**: 
   - Frontend: http://localhost:3000
   - API Documentation: http://localhost:8000/docs
   - Consul Dashboard: http://localhost:8500
   - Prometheus: http://localhost:9090
   - Grafana: http://localhost:3001

### 📚 Additional Resources:

- **Documentation**: Check the `docs/` directory for detailed guides
- **Examples**: Review example scripts in the `examples/` directory
- **API Reference**: Visit `/docs` endpoint when API is running

### 🔧 Key Services and Ports:

| Service | Port | Description |
|---------|------|-------------|
| Frontend | 3000 | Next.js dashboard |
| API | 8000 | FastAPI backend |
| PostgreSQL | 5432 | Database |
| Redis | 6379 | Caching |
| NATS | 4222 | Agent messaging |
| Consul | 8500 | Service discovery |
| Ollama | 11434 | LLM inference |
| Prometheus | 9090 | Monitoring |
| Grafana | 3001 | Monitoring dashboard |

The A2A World Platform is now ready for multidisciplinary data analysis, pattern discovery, and agent-based processing of cultural and mythological data.